#The UNIFAC Method

For a multicomponent mixture, the activity coefficient is calculated by

$$
\ln \gamma_i = \ln \gamma^c_i + \ln \gamma^r_i 
$$

where $\ln \gamma^c_i$ is the combinatorial part and $\ln \gamma^r_i$ is the residual part.

In [118]:
import numpy as np

In [129]:
# Prausnitz case

# acetone and n-pentane
# CH3, CH2, CH3CO
R = np.array([0.9011, 0.6744, 1.6724])
Q = np.array([0.848, 0.540, 1.488])

# Groups
n0 = np.array([1.0, 0.0, 1.0])
n1 = np.array([2.0, 3.0, 0.0])

x = np.array([0.047, 1.0 - 0.047]) # New composition!
N = np.array([n0, n1]) # acetone and pentane in this ORDER
r = np.array([np.sum(n0 * R), np.sum(n1 * R)])
q = np.array([np.sum(n0 * Q), np.sum(n1 * Q)])

##Combinatorial activity

In [130]:
phi = r * x / np.sum(r * x)
theta = q * x / np.sum(q * x)
zz = 10
l = 0.5 * zz * (r - q) - (r - 1.0) 
ln_gamma_c = np.log(phi / x) + 0.5 * zz * q * np.log(theta / phi) \
           + l - (phi / x) * np.sum(x * l)
gamma_c = np.exp(ln_gamma_c)

In [140]:
print ln_gamma_c
print gamma_c

[-0.05271726 -0.00010177]
[ 0.94864819  0.99989824]


##Residual activity

In [132]:
temperature = 307.0 # [K]

a = np.array([[  0.0,   0.0, 476.4],
              [  0.0,   0.0, 476.4],
              [26.76, 26.76,   0.0]])
psi = np.exp(- a / temperature)

# Reference solution
x_acetone = n0 / np.sum(n0)
x_pentane = n1 / np.sum(n1)

theta_pentane = Q * x_pentane / np.sum(Q * x_pentane)
theta_acetone = Q * x_acetone / np.sum(Q * x_acetone)

theta_ref = np.array([theta_acetone, theta_pentane]).T

A = np.einsum('mi,mk', theta_ref, psi)
B = np.einsum('mi,km', (theta_ref.T/A).T, psi) 
ln_gamma_ref = Q * (1.0 - np.log(A) - B)

print ln_gamma_ref

[[ 0.40890193  0.26038566  0.13891146]
 [ 0.          0.          2.43327686]]


In [133]:
X = np.dot(x, N) / np.sum(N.T * x)
theta = Q * X / np.sum(Q * X)

A = np.einsum('m,mk', theta, psi)
B = np.einsum('m,km', (theta / A), psi) 
ln_gamma = Q * (1.0 - np.log(A) - B)

print ln_gamma

[  1.36769256e-03   8.70936299e-04   2.20700647e+00]


In [134]:
ln_gamma_r = np.sum(N * (ln_gamma - ln_gamma_ref), axis=1)
gamma_r = np.exp(ln_gamma_r)

print ln_gamma_r
print gamma_r

[ 1.66056077  0.00534819]
[ 5.26226091  1.00536252]


In [135]:
ln_gamma = ln_gamma_c + ln_gamma_r
gamma = np.exp(ln_gamma)

print ln_gamma
print gamma

[ 1.6078435   0.00524642]
[ 4.99203431  1.00526021]


In [148]:
obtained_gamma = gamma
expected_gamma = np.array([4.99, 1.005])

print 'Obtained activity is %s' % obtained_gamma
print 'Expected activity is %s' % expected_gamma

assert np.allclose(obtained_gamma, expected_gamma, rtol=1.0e-3), 'Obatined value did not match expected!'

Obtained activity is [ 4.99203431  1.00526021]
Expected activity is [ 4.99   1.005]
